- Feature extraction
    - test at least 3 features per group member
    - add at least one lexical feature to your syntactic features (e.g. lemma, POS tag)
- Generate training and test instances
    - Extract predicates, split sentences into propositions and extract the SRL labels
    - Make sure do not use SRL gold data information as features! (except for the predicate). 

- Train a logistic regression classifier (sklearn)
- Test the classifier

# 1. Features

ideas:
- Syntactic Dependency Features
- POS Tag
- Lemma Features
- Contextual Word Features
- Bag-of-Words Feature
- Distance Features


In [5]:
{
    "tokens": ["What", "if", "Google", "Morphed", "Into", "GoogleOS", "?"],
    "features": [
        {"token": "What", "lemma": "what", "pos_tag": "PRON"},
        {"token": "if", "lemma": "if", "pos_tag": "SCONJ"},
        {"token": "Google", "lemma": "Google", "pos_tag": "PROPN"},
        {"token": "Morphed", "lemma": "morph", "pos_tag": "VERB"},
        {"token": "Into", "lemma": "into", "pos_tag": "ADP"},
        {"token": "GoogleOS", "lemma": "GoogleOS", "pos_tag": "PROPN"},
        {"token": "?", "lemma": "?", "pos_tag": "PUNCT"}
    ],
    "predicate_positions": [3],
    "argument_labels": [0, 0, 1, 0, 0, 1, 0]
}

{'tokens': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?'],
 'features': [{'token': 'What', 'lemma': 'what', 'pos_tag': 'PRON'},
  {'token': 'if', 'lemma': 'if', 'pos_tag': 'SCONJ'},
  {'token': 'Google', 'lemma': 'Google', 'pos_tag': 'PROPN'},
  {'token': 'Morphed', 'lemma': 'morph', 'pos_tag': 'VERB'},
  {'token': 'Into', 'lemma': 'into', 'pos_tag': 'ADP'},
  {'token': 'GoogleOS', 'lemma': 'GoogleOS', 'pos_tag': 'PROPN'},
  {'token': '?', 'lemma': '?', 'pos_tag': 'PUNCT'}],
 'predicate_positions': [3],
 'argument_labels': [0, 0, 1, 0, 0, 1, 0]}

# 2. Generate Training and Test Instances

In [2]:
# extract sentences
data_file = "../data/en_ewt-up-train.conllu"
with open(data_file, 'r', encoding="utf-8") as file:
    
    for line in file:
        line = line.strip()
        print(line)
            

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



6	,	,	PUNCT	,	_	24	punct	24:punct	_	_	_	_	_	_
7	when	when	ADV	WRB	PronType=Int	16	mark	16:mark	_	_	ARGM-TMP	_	_	_
8	we	we	PRON	PRP	Case=Nom|Number=Plur|Person=1|PronType=Prs	16	nsubj	16:nsubj|18:nsubj:xsubj	SpaceAfter=No	_	ARG0	ARG0	_	_
9	,	,	PUNCT	,	_	8	punct	8:punct	_	_	_	_	_	_
10	among	among	ADP	IN	_	14	case	14:case	_	_	_	_	_	_
11	many	many	ADJ	JJ	Degree=Pos	14	amod	14:amod	_	_	_	_	_	_
12	other	other	ADJ	JJ	Degree=Pos	14	amod	14:amod	_	_	_	_	_	_
13	irate	irate	ADJ	JJ	Degree=Pos	14	amod	14:amod	_	_	_	_	_	_
14	guests	guest	NOUN	NNS	Number=Plur	8	nmod	8:nmod:among	SpaceAfter=No	_	_	_	_	_
15	,	,	PUNCT	,	_	16	punct	16:punct	_	_	_	_	_	_
16	asked	ask	VERB	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	24	advcl	24:advcl:when	_	ask.02	V	_	ARGM-TMP	_
17	to	to	PART	TO	_	18	mark	18:mark	_	_	_	_	_	_
18	speak	speak	VERB	VB	VerbForm=Inf	16	xcomp	16:xcomp	_	speak.01	ARG1	V	_	_
19	to	to	ADP	IN	_	21	case	21:case	_	_	_	_	_	_
20	the	the	DET	DT	Definite=Def|PronType=Art	21	det	21:det	_	_	_	_	_	_
21	owner	owner	NO

In [8]:
# extract predicates and arguements
import conllu
data_file = "../data/en_ewt-up-test.conllu"
data = open(data_file, 'r', encoding="utf-8")
annotations = data.read()
type(annotations)

str

In [9]:
print(annotations[:1000])

# newdoc id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200
# sent_id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200-0001
# text = What if Google Morphed Into GoogleOS?
1	What	what	PRON	WP	PronType=Int	0	root	0:root	_	_	_
2	if	if	SCONJ	IN	_	4	mark	4:mark	_	_	_
3	Google	Google	PROPN	NNP	Number=Sing	4	nsubj	4:nsubj	_	_	ARG1
4	Morphed	morph	VERB	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	1	advcl	1:advcl:if	_	morph.01	V
5	Into	into	ADP	IN	_	6	case	6:case	_	_	_
6	GoogleOS	GoogleOS	PROPN	NNP	Number=Sing	4	obl	4:obl:into	SpaceAfter=No	_	ARG2
7	?	?	PUNCT	.	_	4	punct	4:punct	_	_	_

# sent_id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200-0002
# text = What if Google expanded on its search-engine (and now e-mail) wares into a full-fledged operating system?
1	What	what	PRON	WP	PronType=Int	0	root	0:root	_	_	_
2	if	if	SCONJ	IN	_	4	mark	4:mark	_	_	_
3	Google	Google	PROPN	NNP	Number=Sing	4	nsubj	4:nsubj	_	_	ARG0
4	expanded	expand	VERB	

In [10]:
sentences = conllu.parse(annotations)

In [14]:
sentences[0]

TokenList<What, if, Google, Morphed, Into, GoogleOS, ?, metadata={newdoc id: "weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200", sent_id: "weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200-0001", text: "What if Google Morphed Into GoogleOS?"}>